In [1]:
from __future__ import print_function,division,absolute_import
import numpy as np
np.random.seed(1337)

In [2]:
#Now Let's define the model
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,MaxPool2D,Activation,Flatten,BatchNormalization
from keras.optimizers import Adam,Adadelta,RMSprop
from keras.losses import categorical_crossentropy
from keras import utils

Using TensorFlow backend.


In [3]:
#
feature_layers_with_batch_norm = [
    Conv2D(64,3,
           padding='same',
           input_shape=(1,100,100)),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(64,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
  
    BatchNormalization(),
    
    Conv2D(128,3),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(128,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    
    BatchNormalization(),
    

    Conv2D(256,3),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(256,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    
    BatchNormalization(),
    
    
    Conv2D(512,3),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(512,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    
    BatchNormalization(),
    
    
    Flatten(),
]

feature_layers_normal = [
    Conv2D(64,3,
           padding='same',
           input_shape=(1,100,100)),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(64,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
  
    
    
    Conv2D(128,3),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(128,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    
    
    

    Conv2D(256,3),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(256,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    
   
    
    
    Conv2D(512,3),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(512,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    
    
    
    
    Flatten(),
]

classification_layers = [
    Dense(512),
    Activation('elu'),
    Dropout(0.5),
    
    
    Dense(256),
    Activation('elu'),
    Dropout(0.5),
    
    Dense(10),
    Activation('softmax')
]


In [4]:
feature_layer_without_dropout = [
    Conv2D(64,3,
           padding='same',
           input_shape=(1,100,100)),
    Activation('elu'),
    Conv2D(64,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
  
    
    Conv2D(128,3),
    Activation('elu'),
    Conv2D(128,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    
    

    Conv2D(256,3),
    Activation('elu'),
    Conv2D(256,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    
    
    Conv2D(512,3),
    Activation('elu'),
    Conv2D(512,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    
        
    Flatten(),
]
classification_layers_without_dropout = [
    Dense(512),
    Activation('elu'),
    
    
    
    Dense(256),
    Activation('elu'),
    
    
    Dense(10),
    Activation('softmax')
]

In [5]:
model_1 = Sequential(feature_layers_normal+classification_layers)
print(model_1.summary())
model_1 = Sequential(feature_layers_with_batch_norm+classification_layers)
print(model_1.summary())
model_1 = Sequential(feature_layer_without_dropout+classification_layers_without_dropout)
print(model_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 64, 100, 100)      640       
_________________________________________________________________
activation_9 (Activation)    (None, 64, 100, 100)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 64, 100, 100)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 98, 98)        36928     
_________________________________________________________________
activation_10 (Activation)   (None, 64, 98, 98)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 49, 49)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 128, 47, 47)       73856     
__________

In [6]:
dat = np.load('ASL_Train.npz')
trainX,TrainY = dat['arr_0'],dat['arr_1']
trainY = utils.np_utils.to_categorical(TrainY,10)
trainX = trainX/255
trainX = trainX.astype('float32')
trainX = trainX.reshape((trainX.shape[0],1,100,100)).astype('float32')
print(trainX.shape,trainY.shape)

(4800, 1, 100, 100) (4800, 10)


In [7]:
dat = np.load('ASL_Test.npz')
testX,TestY = dat['arr_0'],dat['arr_1']
testY = utils.np_utils.to_categorical(TestY,10)
testX = testX/255
testX = testX.astype('float32')
testX = testX.reshape((testX.shape[0],1,100,100)).astype('float32')
print(testX.shape,testY.shape)

(200, 1, 100, 100) (200, 10)


In [8]:
normal_model = Sequential(feature_layers_normal+classification_layers)
normal_model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['accuracy'])

from keras.callbacks import TensorBoard,ModelCheckpoint
from os.path import isfile
data_aug_weight_file = 'ASL-new-normal-weights_bentchmark.h5'

if (isfile(data_aug_weight_file)):
    model.load_weights(data_aug_weight_file)

checkpoint = ModelCheckpoint(data_aug_weight_file, monitor='acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='./logs-ASL-normal_benchmark', histogram_freq=0,write_graph=True, write_images=True)
callbacks_list=[checkpoint,tensorboard]

hist = normal_model.fit(trainX, trainY, batch_size=24,epochs=50,verbose=1,shuffle=True,validation_data=(testX, testY),callbacks=callbacks_list)

acc = np.array(hist.history['acc']).astype('float32')
np.savetxt('1acc-aslnet-all.txt',acc,delimiter=',')

loss = np.array(hist.history['loss']).astype('float32')
np.savetxt('1loss-aslnet-all.txt',acc,delimiter=',')

val_acc = np.array(hist.history['val_acc']).astype('float32')
np.savetxt('1val-acc-aslnet-all.txt',acc,delimiter=',')

val_loss = np.array(hist.history['val_loss']).astype('float32')
np.savetxt('1val-loss-aslnet-all.txt',acc,delimiter=',')


Train on 4800 samples, validate on 200 samples
Epoch 1/50
4800/4800 [==============================] - 53s - loss: 1.8973 - acc: 0.3308 - val_loss: 1.5789 - val_acc: 0.4500
Epoch 2/50
4800/4800 [==============================] - 50s - loss: 0.7642 - acc: 0.7475 - val_loss: 0.6502 - val_acc: 0.7800
Epoch 3/50
4800/4800 [==============================] - 50s - loss: 0.3017 - acc: 0.9048 - val_loss: 0.6278 - val_acc: 0.8350
Epoch 4/50
4800/4800 [==============================] - 50s - loss: 0.1329 - acc: 0.9571 - val_loss: 0.1717 - val_acc: 0.9550
Epoch 5/50
4800/4800 [==============================] - 50s - loss: 0.1042 - acc: 0.9673 - val_loss: 0.2362 - val_acc: 0.9550
Epoch 6/50
4800/4800 [==============================] - 51s - loss: 0.0863 - acc: 0.9740 - val_loss: 0.1381 - val_acc: 0.9650
Epoch 7/50
4800/4800 [==============================] - 50s - loss: 0.0519 - acc: 0.9840 - val_loss: 0.2204 - val_acc: 0.9500
Epoch 8/50
4800/4800 [==============================] - 50s - loss: 0.0

In [9]:
batch_norm_model = Sequential(feature_layers_with_batch_norm+classification_layers)
batch_norm_model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['accuracy'])

from keras.callbacks import TensorBoard,ModelCheckpoint
from os.path import isfile
data_aug_weight_file = 'ASL-new-normal-weights_bentchmark_batchnorm.h5'

if (isfile(data_aug_weight_file)):
    model.load_weights(data_aug_weight_file)

checkpoint = ModelCheckpoint(data_aug_weight_file, monitor='acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='./logs-ASL-normal_benchmark_batchnorm', histogram_freq=0,write_graph=True, write_images=True)
callbacks_list=[checkpoint,tensorboard]

hist = batch_norm_model.fit(trainX, trainY, batch_size=24,epochs=50,verbose=1,shuffle=True,validation_data=(testX, testY),callbacks=callbacks_list)

acc = np.array(hist.history['acc']).astype('float32')
np.savetxt('1acc-aslnet-all-batchnorm.txt',acc,delimiter=',')

loss = np.array(hist.history['loss']).astype('float32')
np.savetxt('1loss-aslnet-all-batchnorm.txt',acc,delimiter=',')

val_acc = np.array(hist.history['val_acc']).astype('float32')
np.savetxt('1val-acc-aslnet-all-batchnorm.txt',acc,delimiter=',')

val_loss = np.array(hist.history['val_loss']).astype('float32')
np.savetxt('1val-loss-aslnet-all-batchnorm.txt',acc,delimiter=',')



Train on 4800 samples, validate on 200 samples
Epoch 1/50
4800/4800 [==============================] - 53s - loss: 3.2299 - acc: 0.3331 - val_loss: 6.5129 - val_acc: 0.1000
Epoch 2/50
4800/4800 [==============================] - 52s - loss: 0.5098 - acc: 0.8435 - val_loss: 4.2852 - val_acc: 0.3000
Epoch 3/50
4800/4800 [==============================] - 52s - loss: 0.2036 - acc: 0.9360 - val_loss: 0.6215 - val_acc: 0.8150
Epoch 4/50
4800/4800 [==============================] - 53s - loss: 0.1076 - acc: 0.9652 - val_loss: 0.3230 - val_acc: 0.9200
Epoch 5/50
4800/4800 [==============================] - 53s - loss: 0.0744 - acc: 0.9773 - val_loss: 0.4308 - val_acc: 0.9100
Epoch 6/50
4800/4800 [==============================] - 53s - loss: 0.0406 - acc: 0.9881 - val_loss: 0.0431 - val_acc: 0.9800
Epoch 7/50
4800/4800 [==============================] - 53s - loss: 0.0468 - acc: 0.9869 - val_loss: 0.0635 - val_acc: 0.9850
Epoch 8/50
4800/4800 [==============================] - 53s - loss: 0.0

In [8]:
norm_model = Sequential(feature_layer_without_dropout+classification_layers_without_dropout)
norm_model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['accuracy'])

from keras.callbacks import TensorBoard,ModelCheckpoint
from os.path import isfile
data_aug_weight_file = 'ASL-new-normal-weights_vanilla.h5'

if (isfile(data_aug_weight_file)):
    model.load_weights(data_aug_weight_file)

checkpoint = ModelCheckpoint(data_aug_weight_file, monitor='acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='./logs-ASL-normal_vanilla', histogram_freq=0,write_graph=True, write_images=True)
callbacks_list=[checkpoint,tensorboard]

hist = norm_model.fit(trainX, trainY, batch_size=24,epochs=50,verbose=1,shuffle=True,validation_data=(testX, testY),callbacks=callbacks_list)

acc = np.array(hist.history['acc']).astype('float32')
np.savetxt('1acc-aslnet-vanilla.txt',acc,delimiter=',')

loss = np.array(hist.history['loss']).astype('float32')
np.savetxt('1loss-aslnet-vanilla.txt',acc,delimiter=',')

val_acc = np.array(hist.history['val_acc']).astype('float32')
np.savetxt('1val-acc-aslnet-vanilla.txt',acc,delimiter=',')

val_loss = np.array(hist.history['val_loss']).astype('float32')
np.savetxt('1val-loss-aslnet-vanilla.txt',acc,delimiter=',')

Train on 4800 samples, validate on 200 samples
Epoch 1/50
4800/4800 [==============================] - 46s - loss: 1.2938 - acc: 0.5519 - val_loss: 0.7884 - val_acc: 0.7500
Epoch 2/50
4800/4800 [==============================] - 44s - loss: 0.1989 - acc: 0.9379 - val_loss: 0.2430 - val_acc: 0.9300
Epoch 3/50
4800/4800 [==============================] - 44s - loss: 0.0638 - acc: 0.9800 - val_loss: 0.3237 - val_acc: 0.9050
Epoch 4/50
4800/4800 [==============================] - 44s - loss: 0.0336 - acc: 0.9908 - val_loss: 0.1143 - val_acc: 0.9750
Epoch 5/50
4800/4800 [==============================] - 44s - loss: 0.0345 - acc: 0.9915 - val_loss: 0.1700 - val_acc: 0.9700
Epoch 6/50
4800/4800 [==============================] - 44s - loss: 0.0657 - acc: 0.9860 - val_loss: 0.3787 - val_acc: 0.9200
Epoch 7/50
4800/4800 [==============================] - 44s - loss: 0.0193 - acc: 0.9933 - val_loss: 0.2052 - val_acc: 0.9550
Epoch 8/50
4800/4800 [==============================] - 44s - loss: 0.0

In [9]:
batch_norm_model = Sequential(feature_layers_with_batch_norm+classification_layers)
batch_norm_model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['accuracy'])

from keras.callbacks import TensorBoard,ModelCheckpoint,EarlyStopping
from os.path import isfile
data_aug_weight_file = 'ASL-new-normal-weights_bentchmark_batchnorm-ear.h5'

if (isfile(data_aug_weight_file)):
    model.load_weights(data_aug_weight_file)

checkpoint = ModelCheckpoint(data_aug_weight_file, monitor='acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='./logs-ASL-normal_benchmark_batchnorm-ear', histogram_freq=0,write_graph=True, write_images=True)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=0, mode='auto')
callbacks_list=[checkpoint,tensorboard]

hist = batch_norm_model.fit(trainX, trainY, batch_size=24,epochs=50,verbose=1,shuffle=True,validation_data=(testX, testY),callbacks=callbacks_list)

acc = np.array(hist.history['acc']).astype('float32')
np.savetxt('1acc-aslnet-all-batchnorm-ear.txt',acc,delimiter=',')

loss = np.array(hist.history['loss']).astype('float32')
np.savetxt('1loss-aslnet-all-batchnorm-ear.txt',acc,delimiter=',')

val_acc = np.array(hist.history['val_acc']).astype('float32')
np.savetxt('1val-acc-aslnet-all-batchnorm-ear.txt',acc,delimiter=',')

val_loss = np.array(hist.history['val_loss']).astype('float32')
np.savetxt('1val-loss-aslnet-all-batchnorm-ear.txt',acc,delimiter=',')


Train on 4800 samples, validate on 200 samples
Epoch 1/50
4800/4800 [==============================] - 53s - loss: 2.7629 - acc: 0.2342 - val_loss: 2.8157 - val_acc: 0.0900
Epoch 2/50
4800/4800 [==============================] - 52s - loss: 0.9931 - acc: 0.6763 - val_loss: 4.6881 - val_acc: 0.1150
Epoch 3/50
4800/4800 [==============================] - 52s - loss: 0.3129 - acc: 0.8963 - val_loss: 0.8690 - val_acc: 0.7500
Epoch 4/50
4800/4800 [==============================] - 53s - loss: 0.1463 - acc: 0.9498 - val_loss: 0.2281 - val_acc: 0.9100
Epoch 5/50
4800/4800 [==============================] - 53s - loss: 0.0848 - acc: 0.9704 - val_loss: 0.2398 - val_acc: 0.9350
Epoch 6/50
4800/4800 [==============================] - 53s - loss: 0.0613 - acc: 0.9815 - val_loss: 0.1256 - val_acc: 0.9650
Epoch 7/50
4800/4800 [==============================] - 53s - loss: 0.0388 - acc: 0.9869 - val_loss: 0.0746 - val_acc: 0.9800
Epoch 8/50
4800/4800 [==============================] - 53s - loss: 0.0